In [ ]:
"""
Routing Pattern in Agentic AI: Customer Support Use Case
========================================================

This example demonstrates how to route customer queries to specialized
AI agents based on the query type. This is a fundamental pattern in
building agentic AI systems.
"""

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from typing import Dict
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ============================================================================
# Step 1: Create the Router Agent
# ============================================================================

router_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a query classifier. Classify customer queries into one category:
- order_status: Order tracking, delivery, shipping, returns, refunds
- product_info: Product features, specs, availability, pricing
- tech_support: Technical issues, troubleshooting, bugs, errors

Respond with ONLY the category name."""),
    ("user", "{query}")
])

#uses LCEL
router_chain = router_prompt | llm | StrOutputParser()

# ============================================================================
# Step 2: Create Specialized Agent Chains
# ============================================================================

# Order Status Agent
order_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful order support agent. Help customers with order tracking, delivery status, returns, and shipping questions. Be friendly and professional."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# Product Information Agent
product_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a product specialist. Provide accurate information about product features, specifications, availability, and pricing."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# Technical Support Agent
tech_agent = ChatPromptTemplate.from_messages([
    ("system", "You are a technical support specialist. Help troubleshoot issues, provide step-by-step solutions, and explain technical problems clearly."),
    ("user", "{query}")
]) | llm | StrOutputParser()

# ============================================================================
# Step 3: Implement the Routing Logic
# ============================================================================

def route_with_metadata(query: str) -> Dict:
    """Route query and return response with routing metadata."""
    category = router_chain.invoke({"query": query}).strip().lower()
    
    agents = {
        "order": order_agent,
        "product": product_agent,
        "tech": tech_agent,
        "support": tech_agent
    }
    
    # Find matching agent
    agent = tech_agent  # default
    for key, value in agents.items():
        if key in category:
            agent = value
            break
    
    response = agent.invoke({"query": query})
    
    return {
        "query": query,
        "category": category,
        "response": response
    }
# ============================================================================
# Step 4: Test the Routing System
# ============================================================================

result = route_with_metadata("My laptop smells like burning plastic")
print(f"Category: {result['category']}")
print(f"Response: {result['response']}")

Category: tech_support
Response: A burning plastic smell coming from your laptop can be concerning and may indicate a serious issue. Here are some steps to troubleshoot the problem:

### Step 1: Power Off Immediately
1. **Shut Down the Laptop**: If you haven't already, power off your laptop immediately to prevent any potential damage or fire hazard.
2. **Unplug the Power Adapter**: Disconnect the power adapter from the laptop and the wall outlet.

### Step 2: Inspect the Laptop
1. **Check for Visible Damage**: Look for any signs of physical damage, such as melted plastic, burnt components, or smoke. Pay special attention to the vents, keyboard, and ports.
2. **Listen for Unusual Noises**: If you hear any unusual sounds (like buzzing or grinding), this could indicate a hardware failure.

### Step 3: Allow to Cool Down
1. **Let it Cool**: Allow the laptop to cool down for at least 30 minutes before handling it further. This will help prevent burns and allow any internal components to set